Exercise 1 - Build a Calculator Tool
Objective: Create a mathematical calculator tool that can handle complex calculations.

Your task is to create a calculator tool that can perform mathematical operations. This tool should be able to handle expressions like "2 + 3 * 4", "sqrt(16)", and "sin(π/2)".

Instructions:
Create a tool called calculator_tool using the @tool decorator.
The tool should accept a mathematical expression as a string.
Use Python's eval() function carefully (or better yet, use the ast module for safety).
Test your tool with various mathematical expressions.
Add your tool to the tools list and test it with the ReAct agent.
Starter Code:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import BaseMessage, HumanMessage, ToolMessage

from langgraph.graph import StateGraph, END

from typing import (Annotated,Sequence,TypedDict)
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages

from langchain_openai import AzureChatOpenAI
from langchain.tools import tool
import os
import json

In [36]:
import math
import ast
import operator

@tool
def calculator_tool(expression: str) -> str:
    """
    Safely evaluate mathematical expressions.
    
    :param expression: A mathematical expression as a string (e.g., "2 + 3 * 4")
    :return: The result of the calculation
    """
    try:
        # Define safe operators and functions
        safe_operators = {
            ast.Add: operator.add,
            ast.Sub: operator.sub,
            ast.Mult: operator.mul,
            ast.Div: operator.truediv,
            ast.Pow: operator.pow,
            ast.USub: operator.neg,
            ast.UAdd: operator.pos,
        }
        
        safe_functions = {
            'sqrt': math.sqrt,
            'sin': math.sin,
            'cos': math.cos,
            'tan': math.tan,
            'log': math.log,
            'log10': math.log10,
            'exp': math.exp,
            'abs': abs,
            'pi': math.pi,
            'π': math.pi,
            'e': math.e,
        }
        
        def safe_eval(node):
            if isinstance(node, ast.Constant):
                return node.value
            elif isinstance(node, ast.BinOp):
                left = safe_eval(node.left)
                right = safe_eval(node.right)
                return safe_operators[type(node.op)](left, right)
            elif isinstance(node, ast.UnaryOp):
                operand = safe_eval(node.operand)
                return safe_operators[type(node.op)](operand)
            elif isinstance(node, ast.Call):
                func_name = node.func.id
                if func_name in safe_functions:
                    args = [safe_eval(arg) for arg in node.args]
                    return safe_functions[func_name](*args)
                else:
                    raise ValueError(f"Function '{func_name}' is not allowed")
            elif isinstance(node, ast.Name):
                if node.id in safe_functions:
                    return safe_functions[node.id]
                else:
                    raise ValueError(f"Variable '{node.id}' is not allowed")
            else:
                raise ValueError(f"Operation not allowed: {type(node)}")
        
        # Parse and evaluate the expression
        tree = ast.parse(expression, mode='eval')
        result = safe_eval(tree.body)
        return str(result)
        
    except Exception as e:
        return f"Error: {str(e)}"
    pass

# TODO: Add calculator_tool to your tools list
# TODO: Test with the agent: "What's 15% of 250 plus the square root of 144?"

In [37]:
print("2 + 3 * 4: ", calculator_tool.invoke("2 + 3 * 4"))  # Example usage
print("sqrt(16): ", calculator_tool.invoke("sqrt(16)"))  # Example usage
print("sin(π/2): ", calculator_tool.invoke("sin(π/2)"))  # Example usage

2 + 3 * 4:  14
sqrt(16):  4.0
sin(π/2):  1.0


In [38]:
tools=[calculator_tool]
tools_by_name={ tool.name:tool for tool in tools}

In [39]:
# Configure ChatOpenAI for Azure OpenAI
deployment_name = "gpt-4o-mini"  # Replace with your actual deployment name

openai_llm = AzureChatOpenAI(
    azure_deployment=deployment_name,
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    temperature=0.7,
    model_name=deployment_name,
    max_retries=3,
    request_timeout=120,
    max_tokens=1024,
    top_p=0.95   
)


In [40]:
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", """
You are a helpful AI assistant that thinks step-by-step and uses tools when needed.

When responding to queries:
1. First, think about what information you need
2. Use available tools if you need current data or specific capabilities  
3. Provide clear, helpful responses based on your reasoning and any tool results

IMPORTANT: Use plain text formatting only. Do not use LaTeX, markdown math notation, or special formatting like \\[ \\] or $$.

Always explain your thinking process to help users understand your approach.
"""),
    MessagesPlaceholder(variable_name="scratch_pad")
])

# binding the tools to the LLM
model_react = chat_prompt | openai_llm.bind_tools(tools)

# Agent state:


class AgentState(TypedDict):
    """The state of the agent."""
    # add_messages is a reducer
    # See https://langchain-ai.github.io/langgraph/concepts/low_level/#reducers
    messages: Annotated[Sequence[BaseMessage], add_messages]


# Example conversation flow:
state: AgentState = {"messages": []}

# append a message using the reducer properly
state["messages"] = add_messages(state["messages"], [HumanMessage(content="Hi")])
print("After greeting:", state["messages"])

# add another message (e.g. a question)
state["messages"] = add_messages(state["messages"], [HumanMessage(content="What's 15% of 250 plus the square root of 144?")])
print("After question:", (state))

After greeting: [HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}, id='89e91662-d921-4934-9e2e-da736e1cd1b9')]
After question: {'messages': [HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}, id='89e91662-d921-4934-9e2e-da736e1cd1b9'), HumanMessage(content="What's 15% of 250 plus the square root of 144?", additional_kwargs={}, response_metadata={}, id='5ba0beb3-2876-44e9-b630-d3397d1ad8a2')]}


In [44]:
user_query = "What's 15% of 250 plus the square root of 144?"

In [45]:
# 1. initial query processing
dummy_state: AgentState = {
    "messages": [HumanMessage(user_query)]}

response = model_react.invoke({"scratch_pad":dummy_state["messages"]})
dummy_state["messages"]=add_messages(dummy_state["messages"],[response])
print("Initial response:", response)

# 2. tool execution - handle ALL tool calls
if response.tool_calls:
    tool_messages = []
    for tool_call in response.tool_calls:
        tool_result = tools_by_name[tool_call["name"]].invoke(tool_call["args"])
        print(f"Tool result for {tool_call['name']}: {tool_result}")
        
        tool_message = ToolMessage(
            content=json.dumps(tool_result),
            name=tool_call["name"],
            tool_call_id=tool_call["id"]
        )
        tool_messages.append(tool_message)
    
    # Add all tool messages at once
    dummy_state["messages"] = add_messages(dummy_state["messages"], tool_messages)

# 3. processing Results and Next Action
response = model_react.invoke({"scratch_pad": dummy_state["messages"]})
dummy_state['messages'] = add_messages(dummy_state['messages'], [response])

# 4. Final response generation - properly formatted output
print("=" * 80)
print("FINAL RESPONSE:")
print("=" * 80)
if response.content:
    # Use repr() to show the raw string, or just print directly for clean output
    print(response.content)
else:
    print("No content in response")

print("\n" + "=" * 80)
print(f"More tools needed: {bool(response.tool_calls)}")
print("=" * 80)

Initial response: content='To solve the expression "15% of 250 plus the square root of 144", I will break it down into two parts:\n\n1. Calculate 15% of 250.\n2. Calculate the square root of 144.\n3. Add the results of these two calculations together.\n\nLet\'s do the calculations step by step. \n\nFirst, I\'ll calculate 15% of 250, which is equivalent to (15/100) * 250. \n\nNext, the square root of 144 is 12.\n\nFinally, I will add the two results together. \n\nI\'ll perform these calculations now.' additional_kwargs={'tool_calls': [{'id': 'call_fciDc0YLV8v3QI9AQDoorhbn', 'function': {'arguments': '{"expression": "0.15 * 250"}', 'name': 'calculator_tool'}, 'type': 'function'}, {'id': 'call_nM5IXA51cUDJIHU7Eo6G9XK4', 'function': {'arguments': '{"expression": "sqrt(144)"}', 'name': 'calculator_tool'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 178, 'prompt_tokens': 196, 'total_tokens': 374, 'completion_tokens_details': {'accepted_prediction_tokens': 0,

this was just for fun, 

let's do it now with Graphs.

first definition of the nodes of the graph:


In [42]:
# tool execution node
def tool_node(state: AgentState):
    """Execute all tool calls from the last message in the state."""
    outputs = []
    for tool_call in state["messages"][-1].tool_calls:
        tool_result = tools_by_name[tool_call["name"]].invoke(tool_call["args"])
        outputs.append(
            ToolMessage(
                content=json.dumps(tool_result),
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
            )
        )
    return {"messages": outputs}


# model invocation node
def call_model(state: AgentState):
    """Invoke the model with the current conversation state."""
    response = model_react.invoke({"scratch_pad": state["messages"]})
    return {"messages": [response]}


# decision logic node
def should_continue(state: AgentState):
    """Determine whether to continue with tool use or end the conversation."""
    messages = state["messages"]
    last_message = messages[-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"

then the graph itself:


In [43]:
# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Add edges between nodes
workflow.add_edge("tools", "agent")  # After tools, always go back to agent

# Add conditional logic
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "tools",  # If tools needed, go to tools node
        "end": END,          # If done, end the conversation
    },
)

# Set entry point
workflow.set_entry_point("agent")

# Compile the graph
graph = workflow.compile()

run/test it

In [46]:
def print_stream(stream):
    """Helper function for formatting the stream nicely."""
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

inputs = {"messages": [HumanMessage(content=user_query)]}

print_stream(graph.stream(inputs, stream_mode="values"))

================================ Human Message =================================

What's 15% of 250 plus the square root of 144?
================================== Ai Message ==================================

To solve the expression "15% of 250 plus the square root of 144," I will break it down into two parts:

1. Calculate 15% of 250.
2. Calculate the square root of 144.
3. Add the two results together.

First, let's calculate 15% of 250:
15% of 250 = 0.15 * 250.

Next, the square root of 144 is 12.

Now, I will perform the calculations. 

Let me calculate 15% of 250 and then add the square root of 144.
Tool Calls:
  calculator_tool (call_6irujZm6ftnMm4X27BpuLjHF)
 Call ID: call_6irujZm6ftnMm4X27BpuLjHF
  Args:
    expression: 0.15 * 250
  calculator_tool (call_bqHJzO71Zpku5W0fkOPq272k)
 Call ID: call_bqHJzO71Zpku5W0fkOPq272k
  Args:
    expression: sqrt(144)
================================= Tool Message =================================
Name: calculator_tool

"12.0"
==============